In [ ]:
from TimeDelay_Neuron_DDF_GaussianForm-Copy1 import *
import numpy as np
import time
import matplotlib.pyplot as plt
import random
import copy
random.seed()
np.random.seed()

In [ ]:
# modify this
use_neuron_index = 3
neuron_txt_filename = 'Episode_2_voltage.txt'
epoch = None # also called "episode". set to None if not specified

In [ ]:
# ======== do not modify below ==========
neuron_name_list = ['Neuron_52_6-8-16',
                    'Neuron_57_6-8-16',
                    'Neuron_61_6-8-16',
                    'Neuron_8-15-2019']
directories_list = ['HVC_ra_x_i_data_2016_2019/50 KhZ Recordings - 06_08_16/50KhZ-06_08_16/50KhZ-06_08_16/Neuron 52/',
                   'HVC_ra_x_i_data_2016_2019/50 KhZ Recordings - 06_08_16/50KhZ-06_08_16/50KhZ-06_08_16/Neuron 57/',
                   'HVC_ra_x_i_data_2016_2019/50 KhZ Recordings - 06_08_16/50KhZ-06_08_16/50KhZ-06_08_16/Neuron 61/',
                   'HVC_ra_x_i_data_2016_2019/08-15-2019/']

Voltage_train = np.loadtxt(directories_list[use_neuron_index]+'Episode_2_voltage.txt')
Current_train = np.loadtxt(directories_list[use_neuron_index]+'Episode_2_current.txt')
Voltage_test = np.loadtxt(directories_list[use_neuron_index]+'Episode_3_voltage.txt')
Current_test = np.loadtxt(directories_list[use_neuron_index]+'Episode_3_current.txt')
plt.figure()
plt.title("Voltage")
plt.plot(Voltage_train, color='blue')
plt.plot(Voltage_test, color='orange')
plt.show()
plt.figure()
plt.title("Current")
plt.plot(Current_train, color='blue')
plt.plot(Current_test, color='orange')
plt.xlim((0,150000))
plt.show()

In [ ]:
tau = 4 # size of time delay in number of timesteps
D = 5 # number of time delay dimensions
length = 110000 # number of timesteps to use for training
Xdata = Voltage_train # data used for training
TT = 0.02 # size of a single timestep in real time (measured in ms, so 0.02 is 0.02ms)
NoCenters_no_thresh = 300 # number of centers to use
DDF = Gauss() # create a Gauss object from the other script
# Combine centers above threshold with centers determined by kmeans
Centers_k_means = DDF.KmeanCenter(Xdata,NoCenters_no_thresh,D,length,tau); # determine center locations using k means
print("C_km:"+str(Centers_k_means.shape))
Center = Centers_k_means
NoCenters = np.shape(Center)[0]
np.savetxt('centers/Center '+neuron_name_list[use_neuron_index]+'.txt',Center)
Center = np.loadtxt('centers/Center '+neuron_name_list[use_neuron_index]+'.txt')
# Center = np.load('Example/Centers_train25k_5k_D'+str(D)+'_tau'+str(tau)+'.npy')

In [ ]:
stim_train = Current_train # Given stimulus during prediction
Pdata = Voltage_test # Truth data which you will compare against prediction
PreLength = 180000 # Prediction Length
beta_arr,R_arr = np.logspace(-10,3,13).tolist(),np.logspace(-10,3,13).tolist()#,[10e-3,10e-2,10e-1,10e0,10e1,10e2,10e3]
bias = 50 # should be larger than tau*(D-1) or something like that
X = np.arange(bias,bias+PreLength*TT,TT) # just for plotting x-axis, not simulation
stim_test = Current_test

In [ ]:
for beta in beta_arr:
    for R in R_arr:
        # beta = 10e0
        # R = 10e-4
        title = neuron_name_list[use_neuron_index]+' with 0.02 time step, D = '+str(D)+', Beta = '+str("{:.1e}".format(beta))+', R = '+str("{:.1e}".format(R))+' Train Time = '+str(length)+', Centers = '+str(NoCenters)+', tau = '+str(tau)
        F = DDF.FuncApproxF(Xdata,length,Center,beta,R,D,stim_train,tau)
        PredValidation = DDF.PredictIntoTheFuture(F,PreLength,stim_test[bias-1:],Pdata[bias-1-(D-1)*tau:])
        # Tau8
        plt.figure(figsize=(20,10))
        plt.plot(X,Pdata[bias:bias + PreLength],label = 'True Voltage', color = 'black')
        plt.plot(X,PredValidation[tau*(D-1)+1:tau*(D-1)+PreLength+1],label = 'Prediction', color = 'r')
        plt.xlabel('time (ms)',fontsize=20)
        plt.ylabel('Voltage',fontsize=20)
        plt.legend()
        plt.title(title,fontsize=20)
        #plt.savefig('Validation Prediction Result')
        plt.savefig(title+'.png')
        plt.show

In [ ]:
beta = 10e2
R = 10e-3
title = neuron_name_list[use_neuron_index]+' with 0.02 time step, D = 3, Beta = '+str("{:.1e}".format(beta))+', R = '+str("{:.1e}".format(R))+' Train Time = 50000, Centers = 1000, tau = '+str(tau)
print(R)
F = DDF.FuncApproxF(Xdata,length,Center,beta,R,D,stim_train,tau)
PredValidation = DDF.PredictIntoTheFuture(F,PreLength,stim_test[bias-1:],Pdata[bias-1-(D-1)*tau:])
# Tau8
plt.figure(figsize=(20,10))
plt.plot(X,Pdata[bias:bias + PreLength],label = 'True Voltage', color = 'black')
plt.plot(X,PredValidation[tau*(D-1)+1:tau*(D-1)+PreLength+1],label = 'Prediction', color = 'r')
plt.xlabel('time (ms)',fontsize=20)
plt.ylabel('Voltage',fontsize=20)
plt.legend()
plt.title(title,fontsize=20)
#plt.savefig('Validation Prediction Result')
plt.savefig(title+'.png')
plt.show

In [ ]:
plt.plot(Voltage_train[:100000])
plt.plot(Pdata[0:180000])
plt.show()

plt.figure()
plt.plot(np.sort(DDF.W))
plt.title("RBF Coefficients (Sorted)")
plt.xlabel('Index (Sorted)')
plt.ylabel("RBF Coefficient Value")
plt.show()

plt.figure()
plt.plot(np.sort(Center[:,0]))
plt.title("Centers")
plt.xlabel("Sorted centers index")
plt.ylabel("Voltage")
plt.show()

plt.scatter(Center[:,0],DDF.W[:-1])
plt.title("Weight as a function of center voltage (unsure)")
plt.xlabel("Center voltage")
plt.ylabel("Weight (Coeff of RBF)")
plt.show()

In [ ]:
plt.show()
plt.figure()
plt.plot(np.sort(Center[:,0]))
plt.title("Voltage of centers")
plt.show()

plt.show()
plt.figure()
plt.plot(Voltage_train[10550:11200])
plt.title("Demonstration of training voltage resolution around spikes")
plt.show()

In [ ]:
print("{:.1e}".format(15.000002))